# Lab 3

## Reading CSV files

In [1]:
import pandas as pd

training_mice = pd.read_csv('./pw3_data/EEG_mouse_data_1.csv')
# training_mice2 = pd.read_csv('./pw3_data/EEG_mouse_data_2.csv')
testing_mice = pd.read_csv('./pw3_data/EEG_mouse_data_test.csv')



## Choosing features

In [3]:

feature_list = [
    "amplitude_around_1_Hertz",
    "amplitude_around_2_Hertz",
    "amplitude_around_3_Hertz",
    "amplitude_around_4_Hertz",
    "amplitude_around_5_Hertz",
    "amplitude_around_6_Hertz",
    "amplitude_around_7_Hertz",
    "amplitude_around_8_Hertz",
    "amplitude_around_9_Hertz",
    "amplitude_around_10_Hertz",
    "amplitude_around_11_Hertz",
    "amplitude_around_12_Hertz",
    "amplitude_around_13_Hertz",
    "amplitude_around_14_Hertz",
    "amplitude_around_15_Hertz",
    "amplitude_around_16_Hertz",
    "amplitude_around_17_Hertz",
    "amplitude_around_18_Hertz",
    "amplitude_around_19_Hertz",
    "amplitude_around_20_Hertz",
    "amplitude_around_21_Hertz",
    "amplitude_around_22_Hertz",
    "amplitude_around_23_Hertz",
    "amplitude_around_24_Hertz",
    "amplitude_around_25_Hertz",
]

input_training_mice = training_mice[feature_list]
output_training_mice = training_mice['state']

#training_mice2 = training_mice2[feature_list]
#testing_mice = testing_mice[feature_list]
print(input_training_mice.head())

   amplitude_around_1_Hertz  amplitude_around_2_Hertz  \
0                  0.000009                  0.000014   
1                  0.000010                  0.000010   
2                  0.000011                  0.000077   
3                  0.000003                  0.000031   
4                  0.000010                  0.000011   

   amplitude_around_3_Hertz  amplitude_around_4_Hertz  \
0                  0.000013                  0.000014   
1                  0.000011                  0.000015   
2                  0.000034                  0.000019   
3                  0.000044                  0.000016   
4                  0.000012                  0.000017   

   amplitude_around_5_Hertz  amplitude_around_6_Hertz  \
0                  0.000023                  0.000009   
1                  0.000022                  0.000017   
2                  0.000013                  0.000006   
3                  0.000007                  0.000006   
4                  0.000020  

## Normalize data

fit and transform all column except "state"  with StandardScaler

In [4]:
from sklearn.preprocessing import StandardScaler
 
scaler = StandardScaler()
for column in input_training_mice:
    
    column_data = input_training_mice[column].to_frame()
    scaler.fit(column_data)
    input_training_mice[column] = scaler.transform(column_data)
        
print(input_training_mice.head())

C:\Users\Jarod\AppData\Local\Temp\ipykernel_19180\3545767515.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_training_mice[column] = scaler.transform(column_data)
C:\Users\Jarod\AppData\Local\Temp\ipykernel_19180\3545767515.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_training_mice[column] = scaler.transform(column_data)
C:\Users\Jarod\AppData\Local\Temp\ipykernel_19180\3545767515.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try usi

   amplitude_around_1_Hertz  amplitude_around_2_Hertz  \
0                 -0.327947                 -0.077315   
1                 -0.308098                 -0.321495   
2                 -0.278126                  3.694203   
3                 -0.502563                  0.953377   
4                 -0.311206                 -0.234021   

   amplitude_around_3_Hertz  amplitude_around_4_Hertz  \
0                 -0.098345                  0.013140   
1                 -0.222925                  0.113253   
2                  1.387219                  0.370614   
3                  2.062970                  0.187826   
4                 -0.170328                  0.227147   

   amplitude_around_5_Hertz  amplitude_around_6_Hertz  \
0                  0.892218                 -0.179646   
1                  0.812089                  0.632424   
2                  0.086868                 -0.512794   
3                 -0.477586                 -0.521958   
4                  0.637048  

C:\Users\Jarod\AppData\Local\Temp\ipykernel_19180\3545767515.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_training_mice[column] = scaler.transform(column_data)
C:\Users\Jarod\AppData\Local\Temp\ipykernel_19180\3545767515.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_training_mice[column] = scaler.transform(column_data)
C:\Users\Jarod\AppData\Local\Temp\ipykernel_19180\3545767515.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try usi

## Create model and fold

In [5]:
import keras
from keras import layers
from sklearn.model_selection import KFold


keras.utils.set_random_seed(123)
kf = KFold(n_splits=3, shuffle=True)

def create_model():
  # 2 inputs
  # 2 hidden nodes
  # 1 output

  mlp = keras.Sequential([
      layers.Input(25),
      layers.Dense(10, activation="tanh"), # Try different numbers of hidden neurons here (e.g. 2, 4, 8, 32, 128)
      layers.Dense(1, activation="tanh"),
  ])

  # Experiment with hyperparameters here:
  # momentum: [0, 0.8, 0.9, 0.99]
  # learning_rate: [0.1, 0.01, 0.001, 0.0001]
  mlp.compile(
      optimizer=keras.optimizers.SGD(learning_rate=0.01, momentum=0.99),
      loss="mse",
  )

  return mlp

mlp = create_model()
mlp.summary()



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 10)                260       
                                                                 
 dense_1 (Dense)             (None, 1)                 11        
                                                                 
Total params: 271 (1.06 KB)
Trainable params: 271 (1.06 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


## Training

In [6]:
history_list = []
trained_mlp = []

for i, (train_index, test_index) in enumerate(kf.split(input_training_mice)):
  # We need to create a new model everytime otherwise fit will continue previous training
 
  mlp = create_model()
  x_train, x_test = input_training_mice.iloc[train_index], input_training_mice.iloc[test_index]
  y_train, y_test = output_training_mice.iloc[train_index], output_training_mice.iloc[test_index]
  
  
  history = mlp.fit(
      x=x_train, y=y_train,
      validation_data=(x_test, y_test),
      epochs=400
  )

  history_list.append(history)
  trained_mlp.append(mlp)

Epoch 1/400



UnimplementedError: Graph execution error:

Detected at node mean_squared_error/Cast defined at (most recent call last):
  File "c:\Python39\lib\runpy.py", line 197, in _run_module_as_main

  File "c:\Python39\lib\runpy.py", line 87, in _run_code

  File "C:\Users\Jarod\AppData\Roaming\Python\Python39\site-packages\ipykernel_launcher.py", line 18, in <module>

  File "C:\Users\Jarod\AppData\Roaming\Python\Python39\site-packages\traitlets\config\application.py", line 1075, in launch_instance

  File "C:\Users\Jarod\AppData\Roaming\Python\Python39\site-packages\ipykernel\kernelapp.py", line 739, in start

  File "C:\Users\Jarod\AppData\Roaming\Python\Python39\site-packages\tornado\platform\asyncio.py", line 205, in start

  File "c:\Python39\lib\asyncio\base_events.py", line 596, in run_forever

  File "c:\Python39\lib\asyncio\base_events.py", line 1890, in _run_once

  File "c:\Python39\lib\asyncio\events.py", line 80, in _run

  File "C:\Users\Jarod\AppData\Roaming\Python\Python39\site